In [ ]:
# !pip install scaper
# !pip install -e
# !pip install jam.py
# !pip install ffmpeg
# !pip install stempeg

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import scaper
import os
import numpy as np
from pathlib import Path
import glob
import warnings
warnings.filterwarnings('ignore')
import jams

In [25]:
# create foreground folder

fg_folder = r'/content/drive/MyDrive/project/musdb train/foreground'
bg_folder = r'/content/drive/MyDrive/project/musdb train/background'

In [26]:
for folder in os.listdir(fg_folder):
    if folder[0] != '.': #ignore system folder
        file = os.listdir(os.path.join(fg_folder, folder))
        print(f"\n{folder}\tfolder contain {len(file)} audio file:\n")
        for sf in sorted(file)[:5]:
            print(f"\t\t{sf}")
        print("\t\t...")


bass	folder contain 100 audio file:

		001 - ANiMAL - Clinic A.wav
		002 - ANiMAL - Rockshow.wav
		003 - Actions - One Minute Smile.wav
		004 - Al James - Schoolboy Facination.wav
		005 - Angela Thomas Wade - Milk Cow Blues.wav
		...

drums	folder contain 100 audio file:

		001 - ANiMAL - Clinic A.wav
		002 - ANiMAL - Rockshow.wav
		003 - Actions - One Minute Smile.wav
		004 - Al James - Schoolboy Facination.wav
		005 - Angela Thomas Wade - Milk Cow Blues.wav
		...

other	folder contain 100 audio file:

		001 - ANiMAL - Clinic A.wav
		002 - ANiMAL - Rockshow.wav
		003 - Actions - One Minute Smile.wav
		004 - Al James - Schoolboy Facination.wav
		005 - Angela Thomas Wade - Milk Cow Blues.wav
		...

vocals	folder contain 100 audio file:

		001 - ANiMAL - Clinic A.wav
		002 - ANiMAL - Rockshow.wav
		003 - Actions - One Minute Smile.wav
		004 - Al James - Schoolboy Facination.wav
		005 - Angela Thomas Wade - Milk Cow Blues.wav
		...


In [27]:
#test of scaper.
seed = np.random.RandomState(123)
# seed = 120
soundscape_duration = 2.0
sc = scaper.Scaper(duration=soundscape_duration, fg_path=fg_folder, bg_path=bg_folder,random_state=seed)

In [28]:
sc.sr = 44100
sc.n_channels = 1
sc.ref_db = -20

In [29]:
labels = ['bass', 'drums', 'other', 'vocals']
for label in labels:
        sc.add_event(label=('const', label),
             source_file=('choose', []),
             source_time=('uniform', 0, 20),
             event_time=('const', 0),
             event_duration=('const', sc.duration),
             snr=('uniform', -5, 5),
             pitch_shift=('uniform', -2, 2),
             time_stretch=('uniform', 0.8, 1.2))

In [43]:
mixture_audio, mixture_jam, annotation_list,stem_audio_list = sc.generate()

In [47]:
ann = mixture_jam.annotations.search(namespace='scaper')[0]

for event in ann:
    print(f"\n{event.value['label']}:\n")
    print(event.value)


bass:

OrderedDict([('label', 'bass'), ('source_file', '/content/drive/MyDrive/project/musdb train/foreground/bass/048 - We Fell From The Sky - Not You.wav'), ('source_time', 15.37644432939188), ('event_time', 0), ('event_duration', 1.8979646905190701), ('snr', 2.244553248606352), ('role', 'foreground'), ('pitch_shift', 0.4440940427103315), ('time_stretch', 1.0537603834205285)])

drums:

OrderedDict([('label', 'drums'), ('source_file', "/content/drive/MyDrive/project/musdb train/foreground/drums/053 - Actions - Devil's Words.wav"), ('source_time', 11.784983534456757), ('event_time', 0), ('event_duration', 2.0), ('snr', -1.2619857319993355), ('role', 'foreground'), ('pitch_shift', -1.0619484992708554), ('time_stretch', 0.966883979210135)])

other:

OrderedDict([('label', 'other'), ('source_file', '/content/drive/MyDrive/project/musdb train/foreground/other/077 - Lyndsey Ollard - Catching Up.wav'), ('source_time', 1.6670593161977654), ('event_time', 0), ('event_duration', 1.900435338301

In [57]:
from IPython.display import Audio, display
display(Audio(data = mixture_audio.T, rate=sc.sr))

In [56]:
# write audio file to some location after makeing using DSD100 dataset to increase dataset for trining process
import stempeg
# extract the annotation data from the JAMS object
ann = mixture_jam.annotations.search(namespace='scaper')[0]
path= r'/content/drive/MyDrive/project/output'
# iterate over the annotation and corresponding stem audio data
for obs, stem_audio in zip(ann.data, stem_audio_list):
    output_path = path + '//'+obs.value['label']+'.wav'
    print(f"Instrument: {obs.value['label']} at SNR: {obs.value['snr']:.2f}")
    stempeg.write_audio(path=output_path, data=stem_audio, sample_rate=44100, output_sample_rate=44100)#write song
    display(Audio(data=stem_audio.T, rate=sc.sr))

Instrument: bass at SNR: 2.24


Instrument: drums at SNR: -1.26


Instrument: other at SNR: -0.66


Instrument: vocals at SNR: 3.90
